<a href="https://colab.research.google.com/github/markod0925/TellMeAStory/blob/main/TranslateChildrenStory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install required packages and load this dataset from hugging face: ajibawa-2023/Children-Stories-Collection

In [11]:
# prompt: Install required packages and load this dataset from hugging face: ajibawa-2023/Children-Stories-Collection

# !pip install -U datasets argostranslate
!pip install -U datasets sacremoses
from datasets import load_dataset
from datasets import Dataset
from google.colab import userdata

dataset = load_dataset("ajibawa-2023/Children-Stories-Collection")

Now extract the first row of dataset and show it

In [12]:
# prompt: Now extract the first 2 rows of dataset and show their Values
print("First prompt:\n")
print(dataset['train'][0]['prompt'])
print("\n\nFirst response:\n")
print(dataset['train'][0]['text'])

First prompt:

Write an educational story (3-5 paragraphs) targeted at young children using simple words. The story should be inspired from this text snippet: 
“How does a Majority Leader prioritize their policy goals when working with members of their party and those across the aisle, and how do they determine which goals to focus on first?
As an AI language model, I don't have personal beliefs or experiences, but I can provide general information on how a Majority Leader may prioritize policy goals while working with members of their party and those across the aisle. Here are some steps that a Majority Leader may follow:

1. Identify the key policy areas and issues that need to be addressed - Majority Leaders typically have an idea about the priority areas that need attention. They may consider the challenges facing their party, their constituents, or the nation as a whole.

2. Assess the level of support for each policy goal - Majority Leaders need to gauge which policy goals have b

Install deep-translator to use GoogleTranslator

In [13]:
!pip install deep-translator

from deep_translator import GoogleTranslator

translator = GoogleTranslator(source='en', target='it')

# Example
translator.translate("Once upon a time, there was a little dragon.")

"C'era una volta, c'era un piccolo drago."

Install Argos Open Translator

In [14]:
# !pip install argostranslate

Preload Argos Models

In [15]:
# import argostranslate.package
# import argostranslate.translate
# import os
# # os.environ["ARGOS_DEVICE_TYPE"] = 'cuda'
# !export ARGOS_DEVICE_TYPE="cuda"

# from_code = "en"
# to_code = "it"

# # Download and install Argos Translate package
# argostranslate.package.update_package_index()
# available_packages = argostranslate.package.get_available_packages()
# package_to_install = next(
#     filter(
#         lambda x: x.from_code == from_code and x.to_code == to_code, available_packages
#     )
# )
# argostranslate.package.install_from_path(package_to_install.download())

Translate the first row

In [16]:
# # Translate "Hello World!"
# print(argostranslate.translate.translate("Hello World!", from_code, to_code))

In [17]:
# print(argostranslate.translate.translate(dataset['train'][0]['prompt'], from_code, to_code))

In [18]:
# prompt: load the dataset "markod0925/Children-Stories-Collection-Italian" and save it as initial value of translated_dataset

try:
    translated_dataset = load_dataset("markod0925/Children-Stories-Collection-Italian")
    translated_dataset = translated_dataset['train'].to_list()
    start_idx = len(translated_dataset)
except:
    translated_dataset = []
    start_idx = 0

print(start_idx)

README.md:   0%|          | 0.00/309 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.66M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1500 [00:00<?, ? examples/s]

1500


In [19]:
from transformers import MarianMTModel, MarianTokenizer

# device = "cuda"
# device = "cpu"
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_name = 'Helsinki-NLP/opus-mt-tc-big-en-it'
tokenizer = MarianTokenizer.from_pretrained(model_name)
try:
  model = MarianMTModel.from_pretrained(model_name).to("cuda")  # ⬅ GPU here
  device = "cuda"
except:
  model = MarianMTModel.from_pretrained(model_name).to("cpu")
  device = "cpu"

def translate_en_to_it(text):
    try:
      text_translated = translator.translate(text)
      print("Figa")
      return text_translated
    except:
      print("Culo")
      # Tokenize input text
      inputs = tokenizer([text], return_tensors="pt", padding=True, truncation=True).to(device)

      # Generate translation
      translated = model.generate(**inputs)

      # Decode the output
      return tokenizer.decode(translated[0], skip_special_tokens=True)

      # tokens = tokenizer.prepare_seq2seq_batch([text], return_tensors="pt").to("cuda")
      # translated = model.generate(**tokens)
      # return tokenizer.decode(translated[0], skip_special_tokens=True)

translate_en_to_it(dataset['train'][0]['prompt'])


Figa


'Scrivi una storia educativa (3-5 paragrafi) rivolta ai bambini piccoli usando parole semplici. La storia dovrebbe essere ispirata da questo frammento di testo: \n“In che modo un leader di maggioranza dà la priorità ai propri obiettivi politici quando si lavora con i membri del loro partito e quelli attraverso il corridoio e come determinano su quali obiettivi concentrarsi prima?\nCome modello di lingua AI, non ho convinzioni o esperienze personali, ma posso fornire informazioni generali su come un leader di maggioranza può dare la priorità agli obiettivi politici mentre si lavora con i membri del loro partito e quelli attraverso il corridoio. Ecco alcuni passaggi che un leader della maggioranza può seguire:\n\n1. Identificare le aree politiche chiave e le questioni che devono essere affrontate: i leader della maggioranza in genere hanno un\'idea delle aree prioritarie che necessitano di attenzione. Possono considerare le sfide che devono affrontare il loro partito, i loro componenti o

In [ ]:
#for row in dataset['train']:
for i, row in enumerate(dataset['train']):
  if i < start_idx:
    continue  # Skip already translated rows

  translated_row = {}

  # Translate prompt
  prompt_text = row['prompt']
  # translated_row['prompt'] = argostranslate.translate.translate(prompt_text, from_code, to_code)
  translated_row['prompt'] = translate_en_to_it(prompt_text)

  # Translate text
  text_content = row['text']
  # translated_row['text'] = argostranslate.translate.translate(text_content, from_code, to_code)
  translated_row['text'] = translate_en_to_it(text_content)

  translated_dataset.append(translated_row)

  if (i + 1) % 100 == 0 or (i + 1) == len(dataset['train']):
    # Push the translated_dataset variable to HF in the dataset space markod0925/Children-Stories-Collection-Italian
    # Assuming translated_dataset is a list of dictionaries
    translated_hf_dataset = Dataset.from_list(translated_dataset)
    translated_hf_dataset.push_to_hub("markod0925/Children-Stories-Collection-Italian", token=userdata.get('HF_TOKEN'))

# translated_dataset now contains the translated data. You can further process or save it.
# For example, print the first translated row
print("\nFirst translated row:")
translated_dataset[0]

Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

...

Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

README.md: 0.00B [00:00, ?B/s]

Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa
Figa


Push the translated_dataset variable to HF in the dataset space
markod0925/Children-Stories-Collection-Italian

In [ ]:
# # prompt: Push the translated_dataset variable to HF in the dataset space
# # markod0925/Children-Stories-Collection-Italian using the HF_TOKEN

# # Assuming translated_dataset is a list of dictionaries
# from datasets import Dataset
# from google.colab import userdata


# translated_hf_dataset = Dataset.from_list(translated_dataset)
# translated_hf_dataset.push_to_hub("markod0925/Children-Stories-Collection-Italian", token=userdata.get('HF_TOKEN'))